# SKA SDP AA0.5LOW Vis Receive Workflow with CBF-Emulator

Instructions to testing the sdp receive workflow using emulated AA0.5LOW data with tango and sdp interfaces against either the persistent sdp or a custom sdp deployment. This notebook can be executed remotely using binderhub via the following link:

https://sdhp.stfc.skao.int/binderhub/v2/gl/ska-telescope%2Fsdp%2Fska-sdp-notebooks/HEAD

## Tango Device Proxy Interface Intro

The Tango device proxy interface provides interaction to a subarray and it's associated execution block in the form of a state machine. When the device is On, this interface provides a single observable state object:

#### ObsState

* EMPTY
* IDLE
* READY
* SCANNING
* FAULT

Communication to the Tango device is performed via the use of commands and accessors whereby all data is conformant to the sdp schemas available here:

https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp.html


### Interface Schemas

| Command    | Current State | Next State | Input  | Output |
| ---------- | ------------- | ---------- | ------ | ------ |
| AssignRes  |     EMPTY     |   IDLE     | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html | None
| Configure  |     IDLE      |   READY    | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html | None
| Scan       |     READY     |  SCANNING  | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-scan.html | None
| RecvAddrs  |    SCANNING   |  SCANNING  | None | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-recvaddrs.html
| EndScan    |    SCANNING   |   READY    | None | None
| End        |     READY     |   IDLE     | None | None
| ReleaseRes |     IDLE      |   EMPTY    | None | None

Note: The next state transition is not instantaneous and should be waited for before executing another command.

### Execution Block

The tango device monitors exactly 1 execution block that is defined by the assign resources command.

### Processing Block

Individual processes running in an execution block when the tango device has resources assigned and is not in the EMPTY state.

In [ ]:
# Restart Kernel to refresh environment variables
import os
os._exit(00)

In [7]:
development = True
debug = True
shared = False  # use the shared persistent sdp deployment
deploy_sdp = True  # if not shared, also deploy sdp to a custom namespace
dev_sdp = False  # use the latest unreleased sdp

# Install Extra Package

In [3]:
!pip install ska_ser_logging ska-sdp-cbf-emulator pyping
!sudo apt install -y wget iputils-ping

# To perform additional developement operations you must additional provide 
# a kubeconfig file which is by default located at:
# $HOME/.kube/config
# or KUBECONFIG environment variable 
# or can be passed to load_kube_config()

if development:
    # Developer Only Packages

    # Kubernetes
    !pip install kubernetes
    # Alternatively debug using kubectl commands
    !curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
    !sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
    !kubectl --help

    # Helm
    !sudo apt install -y gpg
    !curl https://baltocdn.com/helm/signing.asc | gpg --dearmor | sudo tee /usr/share/keyrings/helm.gpg > /dev/null
    !sudo apt-get install apt-transport-https --yes
    !echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/helm.gpg] https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
    !sudo apt-get update
    !sudo apt-get install helm
    !helm --help

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artefact.skao.int/repository/pypi-internal/simple, https://pypi.org/simple
Reading package lists... Done
Building dependency tree       
Reading state information... Done
iputils-ping is already the newest version (3:20180629-2+deb10u2).
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artefact.skao.int/repository/pypi-internal/simple, https://pypi.org/simple
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1232      0 --:--:-- --:--:-- --:--:--  1221
100 42.9M  100 42.9M    0     0  84.3M      0 --:--:-- --:--:-- --:--:-- 84.3M
kubectl controls the Kubernetes cluster manage

# Connect to SDP

The SDP is a helm chart in https://gitlab.com/ska-telescope/sdp/ska-sdp-integration deployed to a namespace on the kubernetes cluster. `dp-shared` is the name of the persistent SDP namespace. For testing the following script can alternatively deploy to a custom personal or team namespace.

**Note:** When deploying the SDP the installation may take a few minutes.

In [8]:
from tango import DeviceProxy, EventType
import ska_sdp_config
import os
import json
import random
from datetime import date
import logging
import ska_ser_logging

if development:
    import kubernetes
    KUBECONFIG = "/app/config" # or "$HOME/.kube/config"
    k8s_client = kubernetes.client.api_client.ApiClient(configuration=kubernetes.config.load_kube_config(KUBECONFIG))
    k8s_core = kubernetes.client.CoreV1Api()
    k8s_batch = kubernetes.client.BatchV1Api()

if shared:
    # Make sure you connect to the correct Configuration Database
    KUBE_NAMESPACE = "dp-shared"
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
else:
    KUBE_NAMESPACE = "dp-yanda"  # add the namespace you want to connect to here
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
    # deploy the sdp
    if development and deploy_sdp:
        if not dev_sdp:
            !helm repo add ska https://artefact.skao.int/repository/helm-internal
            !KUBECONFIG={KUBECONFIG} helm uninstall persistent-sdp --namespace {KUBE_NAMESPACE} --wait
            !KUBECONFIG={KUBECONFIG} helm upgrade --install persistent-sdp ska/ska-sdp --namespace {KUBE_NAMESPACE} --set helmdeploy.namespace={KUBE_PROC_NAMESPACE} --wait
        else:    
            # Alternative: sdp install from local repo
            !apt install git
            !git clone https://gitlab.com/ska-telescope/sdp/ska-sdp-integration.git --init --recursive
            !KUBECONFIG={KUBECONFIG} make uninstall-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE} --wait
            !KUBECONFIG={KUBECONFIG} make install-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE} --wait
    
# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the predetermined host name
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.{KUBE_NAMESPACE}"
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"

"ska" already exists with the same configuration, skipping
Error: uninstall: Release not loaded: persistent-sdp: release: not found
Release "persistent-sdp" does not exist. Installing it now.
NAME: persistent-sdp
LAST DEPLOYED: Thu Sep 29 06:55:18 2022
NAMESPACE: dp-yanda
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
SKA SDP deployed!

You can connect to the configuration database by running a shell in the
console pod. To start a bash shell, use:

    $ kubectl -n dp-yanda exec -it ska-sdp-console-0 -- bash

and from there you can use the ska-sdp command, e.g.:

    # ska-sdp list -a


In [9]:
# Check SDP deployment
if development and debug:
    !KUBECONFIG={KUBECONFIG} helm list -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_PROC_NAMESPACE}

NAME          	NAMESPACE	REVISION	UPDATED                               	STATUS  	CHART         	APP VERSION
persistent-sdp	dp-yanda 	1       	2022-09-29 06:55:18.58732626 +0000 UTC	deployed	ska-sdp-0.11.2	0.11.2     
NAME                              READY   STATUS      RESTARTS        AGE
databaseds-tango-base-0           1/1     Running     1 (5m17s ago)   5m47s
ska-sdp-console-0                 1/1     Running     0               5m47s
ska-sdp-etcd-0                    1/1     Running     0               5m47s
ska-sdp-helmdeploy-0              1/1     Running     0               5m47s
ska-sdp-lmc-config--1-zfz85       0/1     Completed   0               5m47s
ska-sdp-lmc-controller-0          1/1     Running     0               5m47s
ska-sdp-lmc-subarray-01-0         1/1     Running     0               5m47s
ska-sdp-opinterface-0             1/1     Running     0               5m47s
ska-sdp-proccontrol-0             1/1     Running     0               5m47s
ska-sdp-scripts-config--

In [10]:
# tango device
d = DeviceProxy('test-sdp/subarray/01')
d.set_logging_level(5)

# sdp config
config = ska_sdp_config.Config()

# ska logging
logger = logging.getLogger(__name__)
ska_ser_logging.configure_logging(level=logging.INFO)

In [11]:
# Utilities
from tango import DevState
import pytest
import time
import datetime
from typing import Optional

TIMEOUT = 60.0  # seconds
INTERVAL = 0.5  # seconds

def wait_for_predicate(
    predicate, description, timeout=TIMEOUT, interval=INTERVAL
):
    """
    Wait for predicate to be true.

    :param predicate: callable to test
    :param description: description to use if test fails
    :param timeout: timeout in seconds
    :param interval: interval between tests of the predicate in seconds

    """
    start = time.time()
    while True:
        if predicate():
            break
        if time.time() >= start + timeout:
            raise TimeoutError("{description} not achieved after {timeout} seconds")
        time.sleep(interval)


def wait_for_state(device, state, timeout=TIMEOUT):
    """
    Wait for device state to have the expected value.

    :param device: device client
    :param state: the expected state
    :param timeout: timeout in seconds

    """

    def predicate():
        return device.state() == state

    description = f"Device state {state.name}"
    logger.info(f"Waiting for device state {state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)


def wait_for_obs_state(device, obs_state, timeout=TIMEOUT):
    """
    Wait for obsState to have the expected value.

    :param device: device proxy
    :param obs_state: the expected value
    :param timeout: timeout in seconds
    """

    def predicate():
        return device.obsState == obs_state

    description = f"obsState {obs_state.name}"
    logger.info(f"Waiting for device obs_state {obs_state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)

def tango_safe_release(device):
    """
    Safely releases tango device to EMPTY obsState
    """
    if device.obsState == device.obsState.SCANNING:
        logger.info(">> End Scan")
        device.EndScan()
        wait_for_obs_state(device, device.obsState.READY)

    if device.obsState == device.obsState.READY:
        logger.info(">> End")
        device.End()
        wait_for_obs_state(device, device.obsState.IDLE)

    if device.obsState == device.obsState.IDLE:
        logger.info(">> Releasing Resources")
        device.ReleaseResources()        
        wait_for_obs_state(device, device.obsState.EMPTY)
        
    if device.obsState == device.obsState.FAULT:
        device.Restart()
        wait_for_obs_state(device, device.obsState.EMPTY)
        
    assert device.obsState == device.obsState.EMPTY
    logger.info("Tango Device is EMPTY")
    
def tango_safe_off(device):
    """
    Safely turns tango device to OFF state
    """
    tango_safe_release(device)
    if device.state() == DevState.ON:
        logger.info(">> Device OFF")
        device.Off()
        wait_for_state(device, DevState.OFF)

    assert device.state() == DevState.OFF
    logger.info("Tango Device is OFF")

# Debug
def pod_status(namespace: str, name: str) -> Optional[str]:
    pods = k8s_core.list_namespaced_pod(namespace=namespace)
    for i in pods.items:
        if name == i.metadata.name:
            return i.status.phase
    return None

def print_pods(namespace: str):
    pods = k8s_core.list_namespaced_pod(namespace=namespace)
    print(f"pod_ip\t\tnamespace\tname\t\t\t\t\t\tphase")
    for i in pods.items:
        print(f"{str(i.status.pod_ip):10}\t{i.metadata.namespace}\t{i.metadata.name:20}\t{i.status.phase}")

def print_pod_events(namespace: str, name: str):
    field_selector = f"involvedObject.name={name}"
    stream = kubernetes.watch.Watch().stream(k8s_core.list_namespaced_event, namespace=namespace, field_selector=field_selector, timeout_seconds=1)
    print(f"Events for {receive_pod_name}:")
    print("Type\tReason\t\t\tAge\t\tFrom\t\t\tMessage")
    for event in stream:
        print(
            f"{event['object'].type}\t"
            f"{event['object'].reason}\t"
            f"{event['object'].event_time}\t"
            f"{event['object'].reporting_component}\t"
            f"{event['object'].message}"
        )

def print_job_logs(namespace: str, name: str, container: str):
    label_selector = f"job-name={name}"
    pods = sorted(
        k8s_core.list_namespaced_pod(namespace=namespace, label_selector=label_selector).items,
        key=lambda p: p.metadata.creation_timestamp
    )
    pod_name = pods[-1].metadata.name
    print("Logs for:", pod_name)
    print(k8s_core.read_namespaced_pod_log(namespace=KUBE_PROC_NAMESPACE, name=pod_name, container=container))

In [12]:
tango_safe_off(d)

1|2022-09-29T07:01:12.936Z|INFO|MainThread|tango_safe_release|3396606870.py#90||Tango Device is EMPTY
1|2022-09-29T07:01:12.937Z|INFO|MainThread|tango_safe_off|3396606870.py#103||Tango Device is OFF


# Configure Execution Block

To create an execution block, parameters need to be passed to the workflow script and helm charts in ska-sdp-scripts to ensure correct reception is configured.

In [13]:
# list of available workflows
!ska-sdp list script

Keys with prefix /script: 
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-realtime:0.4.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:test-receive-addresses:0.5.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1
/script/realtime:vis-receive:0.6.0


In [14]:
# Config Globals
total_channels      = 13824
total_streams       = 4
rate                = 10416667  # bits per second
channels_per_stream = total_channels // total_streams

total_timesteps     = 6
num_repeats         = 3


def create_receive_parameters(eb_id: str) -> dict:
    max_payloads        = total_timesteps * total_streams * num_repeats
    max_payload_misses  = 30  # payload timeout in seconds
    max_ms              = 1  # -1 to continuously run
    
    return {
        "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
        "version": "3.3.0",
        "pvc": {
            "name": "shared"
        },
        "reception": {
            "num_channels": total_channels,
            "channels_per_stream": channels_per_stream,
            "execution_block_id": eb_id, # alternatives are schedblock filename or datamodel filename
            "layout": "http://127.0.0.1:80/model/default/ska1_low/layout",
            "sdp_config_backend": "etcd3",
            "sdp_config_host": os.environ["SDP_CONFIG_HOST"],
            "sdp_config_port": 2379,
            "continuous_mode": True,
            "transport_protocol": "udp"
        },
        "plasmaEnabled": True,
        "plasma_parameters": {
            "extraContainers": [
                {
                    "name": "tmlite-server",
                    "image": "artefact.skao.int/ska-sdp-tmlite-server:0.3.0"
                }
            ]
        }
    }


def create_resources_config():
    generator = "notebook"
    today = date.today().strftime("%Y%m%d")
    number = random.randint(0, 99998)

    EXECUTION_BLOCK_ID = f"eb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_SENDER = f"pb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_RECEIVER = f"pb-{generator}-{today}-{number+1:05d}"
    PROCESSING_BLOCK_ID_BATCH = f"pb-{generator}-{today}-{number+2:05d}"

    return {
      "interface": "https://schema.skao.int/ska-sdp-assignres/0.3",
      "eb_id":f"{EXECUTION_BLOCK_ID}",
      "max_length": 21600.0,
      "resources": {  # also required in 0.3
        "csp_links": [1, 1],
        "receive_nodes": 1,
        "receptors":["C1", "C2", "C3", "C4"]
      },
      "scan_types": [
        {
          "scan_type_id": "science_a",
          "coordinate_system": "ICRS", "ra": "02:42:40.771", "dec": "-00:00:47.84",
          "channels": [
            { "count": 10, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        },
        {
          "scan_type_id": "science_b",
          "coordinate_system": "ICRS", "ra": "12:29:06.699", "dec": "02:03:08.598",
          "channels": [
            { "count": 5, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        }
      ],
      "channels": [
        {
            "channels_id": "vis_channels",
            "spectral_windows": [
                {
                    "spectral_window_id": "all_channels",
                    "count": 4, "start": 0, "stride": 2,
                    "freq_min": 0.35e9, "freq_max": 0.368e9,
                    "link_map": [[0,0], [200,1], [744,2], [944,3]]
                }
            ]
        }
      ],
      "processing_blocks": [
          {
            "pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_RECEIVER}",
            "workflow": {"kind": "realtime", "name": "vis-receive", "version": "0.5.1"},
            "parameters": create_receive_parameters(EXECUTION_BLOCK_ID)
          }
      ]
    }

# Install and run CBF-Emulator with data from Jupyter

The simplest approach for synchronizing scans between sender and receiver is to run the cbf-emulator packetizer from the runtime controlling the tango device. 

In [15]:
# Download Data
!wget -qnc https://gitlab.com/ska-telescope/sdp/ska-sdp-realtime-receive-core/-/raw/main/data/AA05LOW.ms.tar.gz
!tar -xzf AA05LOW.ms.tar.gz

import cbf_sdp.packetiser
from realtime.receive.core.config import create_config_parser

async def cbf_scan(target_host: str, target_port: str, scan_id: int):
    # Same as:
    #!emu-send\
    #-o transmission.method=spead2_transmitters\
    #-o transmission.channels_per_stream={channels_per_stream}\
    #-o transmission.rate={rate}\
    #-o transmission.target_host={host}\
    #-o transmission.target_start_port={port}\
    #-o reader.num_repeats={num_repeats}\
    #AA05LOW.ms

    sender_args = create_config_parser()
    sender_args['reader'] = {
        'scan_ids': [scan_id],
        'num_repeats': num_repeats
    }
    sender_args['transmission'] = {
        'method': 'spead2_transmitters',
        'channels_per_stream': channels_per_stream,
        'rate': rate,
        'target_host': target_host,
        'target_port': target_port
    }
    await cbf_sdp.packetiser.packetise(sender_args, "AA05LOW.ms")    


# Send-Receive Emulated Scans

Using the cbf-emulator library and tango device, emulated scan reception can be performed by configuring the tango device into the SCANNING state and calling the cbf_scan function to send packets directly from notebook to receiver pod.

In [18]:
import sys
import socket

tango_safe_release(d)
try:
    if d.state() == DevState.OFF:
        print(">> Device ON")
        d.On()
        wait_for_state(d, DevState.ON)
        wait_for_obs_state(d, d.obsState.EMPTY)

    print(">> Assigning Resources")
    config_eb = create_resources_config()
    # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html
    d.AssignResources(json.dumps(config_eb))
    wait_for_obs_state(d, d.obsState.IDLE, timeout=30)
    
    # patch ebconfig for pre vis-receive:0.6.0
    if config_eb["interface"] == "https://schema.skao.int/ska-sdp-assignres/0.3":
        for txn in config.txn():
            eb = txn.get_execution_block(config_eb["eb_id"])
            eb["resources"] = config_eb["resources"]
            txn.update_execution_block(config_eb["eb_id"], eb)

    print(">> Get Receive Address")
    receiveAddresses = json.loads(d.receiveAddresses)
    
    # use scan types from tango response
    scan_types = list(filter(lambda v: v != "interface", receiveAddresses.keys()))
    hosts = [receiveAddresses[scan_type]['host'][0][1] for scan_type in scan_types]
    
    # check all hosts are receiving
    for host in set(hosts):
        found = False
        for tries in range(10):
            try:
                socket.gethostbyname(host)
                found = True
                break
            except socket.gaierror as e:
                logger.info(f"waiting for host {host}")
                time.sleep(5)
        if not found:
            raise TimeoutError(host)
    print("Hosts found")
    
    # perform two scans for each configured scan type
    scan_id = 0
    for scan_type in scan_types:
        host = receiveAddresses[scan_type]["host"][0][1]
        start_port = receiveAddresses[scan_type]["port"][0][1]
        
        print(">> Configure", scan_type)
        # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html
        d.Configure(json.dumps({"interface": "https://schema.skao.int/ska-sdp-configure/0.3", "scan_type": scan_type}))
        wait_for_obs_state(d, d.obsState.READY)

        for scan in range(2):
            print(">> Scan", scan_id)
            d.Scan(json.dumps({"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": scan_id}))
            wait_for_obs_state(d, d.obsState.SCANNING)

            time.sleep(10)
            await cbf_scan(host, start_port, scan_id)
            time.sleep(10)

            print(">> End Scan")

            d.EndScan()
            wait_for_obs_state(d, d.obsState.READY)
            scan_id += 1

        print(">> End")
        d.End()
        wait_for_obs_state(d, d.obsState.IDLE)

    print(">> Releasing Resources")
    d.ReleaseResources()
    wait_for_obs_state(d, d.obsState.EMPTY)

    print(">> Device OFF")
    d.Off()
    wait_for_state(d, DevState.OFF)

except Exception as e:
    # In case of failure, safely restore the device back to its original off state
    tango_safe_release(d)
    raise e

1|2022-09-29T07:06:59.015Z|INFO|MainThread|tango_safe_release|3396606870.py#90||Tango Device is EMPTY
>> Assigning Resources
1|2022-09-29T07:06:59.183Z|INFO|MainThread|wait_for_obs_state|3396606870.py#63||Waiting for device obs_state IDLE...
>> Get Receive Address
1|2022-09-29T07:07:03.232Z|INFO|MainThread|async-def-wrapper|234753785.py#44||waiting for host proc-pb-notebook-20220929-83784-receive-0.receive.dp-yanda-p.svc.cluster.local
1|2022-09-29T07:07:08.240Z|INFO|MainThread|async-def-wrapper|234753785.py#44||waiting for host proc-pb-notebook-20220929-83784-receive-0.receive.dp-yanda-p.svc.cluster.local
1|2022-09-29T07:07:13.250Z|INFO|MainThread|async-def-wrapper|234753785.py#44||waiting for host proc-pb-notebook-20220929-83784-receive-0.receive.dp-yanda-p.svc.cluster.local
1|2022-09-29T07:07:18.259Z|INFO|MainThread|async-def-wrapper|234753785.py#44||waiting for host proc-pb-notebook-20220929-83784-receive-0.receive.dp-yanda-p.svc.cluster.local
1|2022-09-29T07:07:23.267Z|INFO|MainThr